In [1]:
from customizer import *
import keras
from keras import layers
import numpy as np
import random
import sys
import pandas as pd

/home/kc3172/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#text = open('short_reviews_shuffle.txt').read()
text=open('testing_text.txt').read()

In [3]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def findStrAfterStr(myString, searchText, afterText):
    after_index = myString.index(afterText)
    return myString.find(searchText, after_index)

def get_second_index(input_string, sub_string):
    return input_string.index(sub_string, input_string.index(sub_string) + 1)

#chars = sorted(list(set(text)))
# Dictionary mapping unique characters to their index in `chars`
chars=['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']

char_indices = dict((char, chars.index(char)) for char in chars)
maxlen=60
step=1

#MODEL
model = keras.models.Sequential()
model.add(layers.LSTM(1024, input_shape=(maxlen, len(chars)),return_sequences=True))
model.add(layers.LSTM(1024, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
model.load_weights("./notebook/saved_weights/May-5-all-01-0.6371.hdf5")

optimizer = keras.optimizers.Adam(lr=0.0002)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


In [4]:
#PRINT RANDOM REVIEWS
# Select a text seed at random
def random_reviews(number_of_characters):
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    tt = []
    print('Coming up with several reviews for you...')

    for temperature in [0.8]:
        sys.stdout.write(generated_text)

        # We generate characters
        for i in range(number_of_characters):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            tt.append(next_char)

            generated_text += next_char
            generated_text = generated_text[1:]

            savedStdout = sys.stdout  # 保存标准输出流
            with open('outv3.txt', 'a') as file:
                sys.stdout = file  # 标准输出重定向至文件
                sys.stdout.write(next_char)

            sys.stdout = savedStdout  # 恢复标准输出流
            sys.stdout.write(next_char)
            sys.stdout.flush()
        
    file = open("reviews_5starv3.txt","w") 
    file.write(''.join(tt)) 
    file.close() 
    print ("finished")

In [5]:
#PRINT USER CUSTOMIZED REVIEWS
def user_custom(foods):
    #foods is a string separated by commas
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('Coming up with two ideas for you...')

    final=generated_text+''

    for temperature in [0.8]:


        # We generate 600 characters
        for i in range(600):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            final+=next_char

            generated_text += next_char
            generated_text = generated_text[1:]
    #print first review, then second via SOR/EOR
    temp=personalized_clean_up(final, foods)
    start=temp.find('SOR')
    stop=findStrAfterStr(temp, 'EOR', 'SOR')
    end_first=temp[start+4:stop]

    new=temp[get_second_index(temp, 'SOR')+4:]
    ending=new.find('EOR')
    print(temp[start+4:stop])
    print("")
    print(new[:ending])

In [6]:
#PRINT dataframe CUSTOMIZED REVIEWS
def df_custom(restaurant):
    #restaurant is a string exactly the same as in the dataframe
    df=pd.read_csv('all_five_star_restaurants.csv')
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('Coming up with two ideas for you...')

    final=generated_text+''

    for temperature in [0.8]:


        # We generate 600 characters
        for i in range(600):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            final+=next_char

            generated_text += next_char
            generated_text = generated_text[1:]
    #print first review, then second via SOR/EOR
    items=pull_restaurant_review(restaurant, df)

    temp=clean_up(final, items)
    start=temp.find('SOR')
    stop=findStrAfterStr(temp, 'EOR', 'SOR')
    end_first=temp[start+4:stop]

    new=temp[get_second_index(temp, 'SOR')+4:]
    ending=new.find('EOR')
    print(temp[start+4:stop])
    print("")
    print(new[:ending])


In [7]:
random_reviews(100000)

Coming up with several reviews for you...
 This file is just something to help the model get started! Amazing!!

/home/kc3172/py3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


!!<EOR>"
"<SOR>We love coming in for lunch. Friendly service, good food and great feel. Would definitely go back.<EOR>"
"<SOR>First time eating here. I loved the food, the manager, was friendly and fast.<EOR>"
"<SOR>The best food I've had in a long time. Steak and lobster, chicken tikka masala, and the perfect flavor to the wall.<EOR>"
"<SOR>Simply delicious food. Great customer service. I had a barbacoa burrito and my mom and i were too full. The service was friendly and courteous. A very good feel of a few other places in the city.<EOR>"
"<SOR>Fantastic food and service.  I'm a big fan of their chicken curry and the breakfast burrito is amazing and the prices are reasonable.   My boyfriend and I try to plan a visit when I want to have a great brunch on the weekend and the brisket and sundaes excellent. This is one of the best restaurants in the area, so when I am in Las Vegas, I met my staple and have a good time.<EOR>"
"<SOR>So damn good! I went with the feel on the menu and they we

"<SOR>My favorite spot for good pizza.  I haven't had a bad experience yet.<EOR>"
"<SOR>The place is great. The food is great and also they have a fish taco that are delicious.  Surprisingly cheap and delicious.   Fresh oysters were excellent.  Even more special on a giant scrambled eggs, perfect crust and the service was friendly and inviting. I actually would eat here everyday. I will never eat beef sandwich.<EOR>"
"<SOR>We had a wonderful time. The food was amazing with good portions. I recommend this place. Try the Green Curry.<EOR>"
"<SOR>Had the jerk chicken burrito with potatoes. It was amazing as well. I will definitely be going back.<EOR>"
"<SOR>Absolutely delicious! I recommend the collard greens and the green curry. The bread is excellent. Friendly staff, clean bathroom and food at a great price. Love it here!<EOR>"
"<SOR>The spicy kick in the pork belly is one of the best I've had. Would recommend this place to all our friends. We have tried several places here and I have t

"<SOR>Great place. Good prices, friendly service.  We got three times in a row while in Vegas and we're fortunate to make sure i came to enjoy good food and service. I paid for all you can eat. But definitely worth the trip.<EOR>"
"<SOR>Went there for my niece, all there were 6 of us and they were wonderful. Highly recommended. Simply the best food I've had in the area. Try the Beach Brioche and the Dominickn gumbo.<EOR>"
"<SOR>Delicious, delicious food, good prices, amazing staff, and our server Jeremy is so knowledgable and attentive, we were in last minute and she was amazing!<EOR>"
"<SOR>Love this place.  Great food.  Even better service.  The bartender was professional and kind and helpful.  I think I'll be back again!<EOR>"
"<SOR>Great lunch specials. All you can eat sushi and 2 of my favorite rooms.<EOR>"
"<SOR>Absolutely love this place! Super courteous and friendly service and amazing food.<EOR>"
"<SOR>The chicken poutine was delicious! We stooded here enough to fit in an open

...

"<SOR>YUMMMMM MEAT.  So yummy. Service was good.  Great casual lunch.<EOR>"
"<SOR>This is my local of do the interior, lots of tv's and lots of choices. Vitamin pho is a must.<EOR>"
"<SOR>The food and service were excellent, my favorite soup dumpling was the best I've had. Good price great friendly service.<EOR>"
"<SOR>The food was good. I had the lamb chops and a few extras. I plan to go back for sure.<EOR>"
"<SOR>Came here after a baseball game and everything was on point! First we had the southwestern egg roll, my children and it was amazing. Server was friendly and professional. Michelle was our party of 10 and they were all so gracious and attentive.  We will definitely be back!<EOR>"
"<SOR>Don't let the neighborhood but one can get really well and comes with a queso ricotta cheese.<EOR>"
"<SOR>The best Korean bbq in town hands down. The Lobster Bisque is off the hook so it's awesome! Staff is friendly and they play variety. Highly recommend it!<EOR>"
"<SOR>Amazing!! Dumplings are

"<SOR>Such a great place we got caestration all day. Pricey but worth it for the quantity.<EOR>"
"<SOR>If you want a breakfast sandwich, some sauces, pickles, chunks of meat and cheese sauce.  Wonderful.  Staff was super friendly.<EOR>"
"<SOR>Everything was delicious.  I had the sirloin and my friend had the mac n cheese and my husband had the chicken and waffles. Both were delicious, the restaurant is small so be prepared for a premium in my book.<EOR>"
"<SOR>I have never had a bad meal here great food. Everything I have tried is good. The hummus is cooked to perfection. Highly recommended!!<EOR>"
"<SOR>The food was great, the spice is bomb! I would definitely recommend this place!<EOR>"
"<SOR>Great food, great service friendly staff. I eat there at least once a week. They always get my order right and the food is always hot and fresh. It is a popular Spot so much but this is in the bar and service is pleasant - thank you for the great service!<EOR>"
"<SOR>Excellent food and service t

"<SOR>The staff were amazing. We had a enchillanan grind with spaghetti with meatball and let me just prepare my order up by the perfect version to giving it a try. The waitress was so sweet and attentive, he knew a lot of food and was very accommodating and friendly . Would recommend to any chill atmosphere.<EOR>"
"<SOR>This is a great addition to Charlotte. I go at least once a week. The staff is very friendly. The food is awesome and the ambience is so relaxing and it costs a little bit away from the strip, we ordered the bean and cheese burro, and gyro.  And the pork bun with the vegetarian options were out of this world delicious. The maple bacon wrap was very good, and the portions were so big, so were the meats were delicious. The servers were so nice and helpful. Definitely coming back.<EOR>"
"<SOR>Delicious food. Pizza was out of this world. Service was quick and friendly. Definitely recommend eating here<EOR>"
"<SOR>The food and service were great. I will make this a regular 

"<SOR>The food is delicious and the staff are so so nice , we love their menu and ingredients are cooked perfectly and on point. Ask for Christina for the great service in this new location.  The food was great too.  We were not disappointed!  Good environment and desserts a good variety of dishes and appetizers were better than period.  Our waitress was funny and super nice.  Had a shrimp pizza and she loved it as well.  awesome people<EOR>"
"<SOR>We had the Sparkling Sandwich so far the waitress was very helpful and friendly. I was here on vacation and had another great meal here. I will definitely be coming back whenever I come back to Vegas.<EOR>"
"<SOR>The fish was very fresh and delicious. The service was blowing with many varieties to choose from. The Shit Martini was amazing & our food served was all very fresh and delicious. Hardest broth is the mango stuffed pizzas and baklava. They also have a drive thre of old towners. So, do yourself a favor......And the burgers are top no

"<SOR>Best burger I've ever had. Amazing mozzy and sweet bread. Great atmosphere and family style. I loved it. The service was great and I will definitely come back. Definitely will be doing anything but he helped us out with an extra star of this place for the food quickly. Beautiful place on pretentious and morning<EOR>"
"<SOR>This place is super good. My favorite is the Dolmade and Chilaquilas with chicken.<EOR>"
"<SOR>Great service.  Odd our nights with a group of 6 and had to try the cracker chicken Shawarma, she took the rise trio and the house salad with rice and salad to go with the taro boba. They also have really good coffee<EOR>"
"<SOR>Great for a romantic dinner. We came here on a Wednesday with the family and the food was some of the best I've had. The inside wasnt up for your business which is nice in the food quality and service was ok.<EOR>"
"<SOR>So good!  Really friendly staff that have a line up for a real meal.<EOR>"
"<SOR>I love this place! I go at least once a wee

In [ ]:
len(text)